## Automate Visiting a URL in a Browser and Typing Stuff and Moving Files

**Visit a website and change name before saving**


In [6]:
!python --version

Python 3.10.12


In [1]:
import pyautogui
import webbrowser
import time
import os
import shutil
import pandas as pd


In [2]:
def move_file_if_not_stored(new_downloaded_file, final_file_location):
    tcx_file_is_stored = True
    if os.path.exists(new_downloaded_file):
        if not(os.path.exists(final_file_location)):
            shutil.move(new_downloaded_file, final_file_location)
            print(f"Moved file into:  {final_file_location}")
            return tcx_file_is_stored
        else:
            print(f"{final_file_location} ALREADY EXISTS")
            return tcx_file_is_stored
    else:
        if os.path.exists(f"{final_file_location}"):
            print(f"{final_file_location} ALREADY EXISTS")
            return tcx_file_is_stored
        else:
            print(f"{new_downloaded_file} WAS NOT DOWNLOADED")
            return False


Setup Location Defaults

In [3]:
workoutExportURL = f"https://www.mapmyfitness.com/workout/export/csv"

filename = '~/Downloads/user2632022_workout_history.csv'

patternstrip = 'http://www.mapmyfitness.com/workout/'

project_folder = '/Users/barbaraihidalgo-sotelo/PROJECTS/build_workout_dashboard'

data_folder = project_folder + '/workouts_timedata'


### 1. Download the latest user_workout_history to get all workout IDs

PRE-REQUISITE:  
**You have to sign in before this will work** 

The cell below opens the export URL in a new tab, waits 15 sec to load the 10+ yrs of workouts

(Optional) If automated, set a wait-period and click enter to save to (last downloaded) save


In [ ]:
webbrowser.open(workoutExportURL)
# time.sleep(15)
# pyautogui.press('enter')


#### Sort workout_summary table if you want, by Workout Time, for example

The idea is that most workouts won't cause the wait-time to timeout, so lets download those first by sorting the table. 

Then, check which workout IDs already exist in a storage folder, and exclude those from the list to download. 


In [4]:
# Read the user_workout_history to get a list of all workout ids
df = pd.read_csv(filename)
df = df[df['Workout Time (seconds)']>1].sort_values(by='Workout Time (seconds)')

all_workouts = df['Link'].str.strip(patternstrip).tolist()

# Check which workout_IDs already exist in the folder and remove those from the list to download
gotem = [id.strip('.tcx') for id in os.listdir(data_folder)]

ids2download = [id for id in all_workouts if id not in gotem]

wait_time = 4

#### Loop through a list of workout IDs and download a renamed file locally

**Use PyAutoGui to load a page**  

The strategy is to visit the export URL in a new browser window, rename it with the workout id and save.

Then, check for the presence of the successfully downloaded file and, if not in the final storage location, move it. 


In [ ]:
# workoutid = '8287240762'
for workoutid in ids2download:
    locationDataURL = f"https://www.mapmyfitness.com/workout/export/{workoutid}/tcx"
    
    webbrowser.open(locationDataURL)
    time.sleep(wait_time)   # 5 sec seems a little long for the short files, but will def be too short for long runs & anomalies
    
    pyautogui.write([l for l in str(workoutid)], interval = 0.2)
    pyautogui.press('enter')
    time.sleep(1)

    # Check for the file and move
    move_file_if_not_stored(f"/Users/barbaraihidalgo-sotelo/Downloads/{workoutid}.tcx" , f"{data_folder}/{workoutid}.tcx" )
